# SAT_Kepler

- Frac
- Elements
- State

In [4]:
using NBInclude
using LinearAlgebra

#@nbinclude("./SAT_Const.ipynb")
@nbinclude("./SAT_VecMat.ipynb")

In [2]:
function Elements(GM, y)
"""
Purpose:
    Computes the osculating Keplerian elements from the satellite state vector
    for elliptic orbits

Input:    
     GM  Gravitational coefficient (gravitational constant * mass of central body)
     u   State vector (x,y,z,vx,vy,vz)
Output:
     Keplerian elements (a,e,i,Omega,omega,M) with
      a      Semimajor axis
      e      Eccentricity
      i      Inclination [rad]
      Omega  Longitude of the ascending node [rad]
      omega  Argument of pericenter  [rad]
      M      Mean anomaly  [rad]
"""

     r=y[1:3]
     v=y[4:6]
     h=cross(r,v)
     H=norm(h)
 
     Omega=atan(h[1],-h[2])
     Omega=mod(Omega,2.0*pi)
     i=atan(sqrt(h[1]*h[1]+h[2]*h[2]),h[3])  
     u=atan(r[3]*H,-r[1]*h[2]+r[2]*h[1])
 
     R=norm(r)
     a=1.0/(2.0/R-dot(v,v)/GM)
 
     eCosE = 1.0-R/a
     eSinE = dot(r,v)/sqrt(GM*a)
 
     e2 = eCosE*eCosE +eSinE*eSinE
     e  = sqrt(e2)
     E  = atan(eSinE,eCosE)

     M  = mod(E-eSinE,2.0*pi)
     nu = atan(sqrt(1.0-e2)*eSinE, eCosE-e2)
     omega = mod(u-nu,2.0*pi)

     return(a,e,i,Omega,omega,M)
    
end

Elements (generic function with 1 method)

In [5]:
function State(GM, Kep, dt)
"""
  Purpose:
    Computes the satellite state vector from osculating Keplerian elements
    for elliptic orbits
    
  Input/Output:
    GM        Gravitational coefficient
    Kep       Keplerian elements (a,e,i,Omega,omega,M) 
    dt        Time since epoch
    <return>  State vector (x,y,z,vx,vy,vz)
"""    

  # Keplerian elements at epoch
  
  a = Kep[1];  Omega = Kep[4];
  e = Kep[2];  omega = Kep[5]; 
  i = Kep[3];  M0    = Kep[6];

  # Mean anomaly

  if (dt==0.0) 
    M = M0
  else 
    n = sqrt(GM/(a*a*a))
    M = M0 +n*dt
  end

  # Eccentric anomaly
  
  E=EccAnom(M,e)   

  cosE = cos(E) 
  sinE = sin(E)

  # Perifocal coordinates

  fac=sqrt((1.0-e)*(1.0+e))  

  R = a*(1.0-e*cosE)   # Distance
  V = sqrt(GM*a)/R     # Velocity

  r = [a*(cosE-e), a*fac*sinE , 0.0]
  v = [-V*sinE, +V*fac*cosE, 0.0] 

  # Transformation to reference system (Gaussian vectors)
  
  A1=zeros(3,3)
  A2=zeros(3,3)
  A3=zeros(3,3)
  R_z(A1,-Omega)
  R_x(A2,-i)
  R_z(A3,-omega)
  PQW = A1 * A2 * A3

  r = PQW*r
  v = PQW*v

  
  return vcat(r,v)

end


State (generic function with 1 method)

In [1]:
function EccAnom(M,e)
"""
  Purpose:
    Computes the eccentric anomaly for elliptic orbits
  Input/Output:
     M         Mean anomaly in [rad]
     e         Eccentricity of the orbit [0,1[
    <return>  Eccentric anomaly in [rad] 
    
"""
  eps_mach=1.e-16
  maxit = 15
  eps = 100.0*eps_mach

  i=0

  # Starting value

  M = mod(M, 2.0*pi)   
    
  if (e<0.8)
       E=M
  else 
       E=pi
  end

  # Iteration

  f=1.0
  while (abs(f) > eps) 
    f = E - e*sin(E) - M
    E = E - f / ( 1.0 - e*cos(E) )
    i+=1
    if (i==maxit)
      println(" convergence problems in EccAnom")
      break
    end
  end       

  return E

end

"""
const eps_mach=1.e-16
EccAnom(100,0.1)
"""


EccAnom (generic function with 1 method)